# Configuration Notebook
Useful for debugging configurations and viewing configuration details.

## Setup

In [ ]:
import sys, os
# Adjust this to match the relative location of the notebook to the 'src' directory
modules_path = os.path.join('..', 'src')
if modules_path not in sys.path: sys.path.insert(0, modules_path)

from pprint import pformat, pp
from IPython import display

from forgather.config import (
    load_config,
    ConfigEnvironment,
    fconfig,
    pconfig
)

from aiws.config import base_preprocessor_globals
import aiws.notebooks as nb

# Defaults
project_directory = "example"
config_template = "example_experiment.yaml"

fc_project_directory = None
fc_config_template = None

## Project
Set 'project_directory' to the path of the project directory.

In [ ]:
# Optional: override default with widget file selector
from ipyfilechooser import FileChooser

fc_project_directory = FileChooser(
    show_only_dirs=True,
    select_desc="Select project directory",
)
display.display(fc_project_directory)

In [ ]:
# Load project meta-configuration
if fc_project_directory is not None:
    project_directory = os.path.relpath(fc_project_directory.selected)
meta_config_path = os.path.join(project_directory, 'meta_config.yaml')
metacfg = load_config(meta_config_path, project_directory=project_directory)
nb.show_project_readme(metacfg)
nb.display_meta_config(meta_config_path, metacfg, "### Meta Config\n")
nb.list_templates(metacfg.experiment_dir, "### Available Configurations\n")

## Configuration
Set 'config_template' to the name of a configuration template in the project.

In [ ]:
# Optional: override default with widget file selector
from ipyfilechooser import FileChooser
fc_config_template = FileChooser(
    path=project_directory,
    filter_pattern=["*.yaml"],
    sandbox_path=project_directory,
    select_desc="Select a configuration to load",
)
display.display(fc_config_template)

In [ ]:
# Load configuration
if fc_config_template is not None:
    config_template_path = os.path.relpath(fc_config_template.selected)
else:
    config_template_path = os.path.join(metacfg.experiment_dir, config_template)
nb.display_filelink(config_template_path, "### Configuration File\n")

# Create configuration envrionment
cfg_environment = ConfigEnvironment(
    searchpath=metacfg.search_paths,
    # Add project directory to base globals
    globals=base_preprocessor_globals() | dict(project_directory=project_directory)
)
nb.display_preprocessed_template(cfg_environment, config_template_path, title="### Preprocessed Configuration\n")
nb.display_referenced_templates_tree(cfg_environment, config_template_path, "### Included Templates\n")

# Load the configuration
loaded_config = cfg_environment.load(config_template_path)
nb.display_referenced_source_list(loaded_config.config, "### Included Sources\n")
display.display(display.Markdown("### Loaded Configuration\n"))
pconfig(loaded_config.config)

## Materialized Configuration

Instantiate the configuration from the definition.

In [ ]:
from aiws.training_loop import TrainingScriptConfig

# Wrap the data in a TrainingScriptConfig
config = TrainingScriptConfig(**loaded_config.config)

# Materialize the trainer
# This config format expects the pre-processed config to be injected, as this
# can be used to log the configuration.
config.trainer = config.trainer(pp_config=loaded_config.pp_config)
pconfig(config)

### Test Configuration

Run the trainer in the notebook.
Note: Running directly from the notebook may have issues.

For a more robust approach, see: [train.ipynb](train.ipynb)

In [ ]:
config.trainer.train()

### Cleanup
Note: These will show the target directory and ask for confirmation before proceeding.

#### Delete All
This will delete all models and logs for the project.

In [ ]:
nb.delete_dir(metacfg.model_dir, "Delete all models in project")

#### Delete Configuration Output Directory
This will delete the model and logs for the current configuration.

In [ ]:
nb.delete_dir(config.output_dir, "Delete output directory")